In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import streamlit as st
import joblib
import statsmodels
from PIL import Image, ImageOps


In [22]:
year_to_forecast = 2028
years_into_the_future = year_to_forecast - 2023

In [23]:
#model calculations
model = joblib.load('wind_generation_SARIMA_model.pkl')

In [24]:
#calculate steps for model to forecast from 2023 onwards
#X3_w_test starts May 2020
steps_rem_2020 = 8
steps_2021_to_2023 = 3*12
steps_test = steps_rem_2020 + steps_2021_to_2023 + (12*(int(year_to_forecast)-2023))

#generate forecast
forecast = model.forecast(steps=steps_test)

#calculate total generation for selected year (MWh)
#data is in half-hourly records; divide by 2 to get MWh
selected_yr_forecasted_wind_mwh = forecast.loc[str(year_to_forecast)].sum() / 2

#calculate TWh
selected_yr_forecasted_wind_twh = selected_yr_forecasted_wind_mwh/1000000

#load .csv of non-wind generation in 2023
non_wind_data = pd.read_csv('energy_gen_2023_excl_wind.csv')

#add wind calculations to dataframe
non_wind_data.loc[len(non_wind_data)] = ['WIND', selected_yr_forecasted_wind_mwh]

#calculate total energy generation
total_generation = non_wind_data['Generation (MWh)'].sum()

#calculate total energy opex
total_cost = selected_yr_forecasted_wind_mwh*67
total_cost_bil = total_cost/1000000000

In [25]:
#UK govt March 2024 strike prices for onshore and offshore wind
w_off_strike_price = 73 #per MWh
w_on_strike_price = 64 #per MWh
w_years_to_build = 0.5

#avg build cost per 50-turbine farm
cost_wind_farm = 617380000 #mil GBP per 50-turbine farm, 2.75MW ea

#wind generation per farm = avg 50 turbines * 4380MWh/yr per turbine
w_mwh_year = 219000 

w_total_farms = selected_yr_forecasted_wind_mwh/w_mwh_year
w_total_capex = w_total_farms * cost_wind_farm
w_total_opex = (w_off_strike_price * selected_yr_forecasted_wind_mwh * 0.66) + (w_on_strike_price * selected_yr_forecasted_wind_mwh * 0.33)
w_total_cost = (w_total_capex + w_total_opex)/1000000000

w_total_cost

436.4867703135205

In [26]:
#SOLAR
s_strike_price = 61 #per MWh
s_years_to_build = 1

#avg build cost per solar farm
cost_solar_farm = 980000 #mil GBP per 20MW farm

#solar generation per farm = 20MW farm * 2146 MWH/year/MW
s_mwh_year = 42920

s_total_farms = selected_yr_forecasted_wind_mwh/s_mwh_year
s_total_capex = s_total_farms * cost_solar_farm
s_total_opex = s_strike_price * selected_yr_forecasted_wind_mwh 
s_total_cost = (s_total_capex + s_total_opex)/1000000000

s_total_cost

12.668688849076535

In [27]:
#NUCLEAR
n_strike_price = 87 #per MWh
n_years_to_build = 12

#avg build cost per nuclear power plant
cost_nuclear_plant = 35000000000 #mil GBP per plant

#nuclear energy generation per 3.2FW plant * 8,000,000 MWh/year/GW
n_mwh_year = 25600000

n_total_plant = selected_yr_forecasted_wind_mwh/n_mwh_year
n_total_capex = n_total_plant * cost_nuclear_plant
n_total_opex = n_strike_price * selected_yr_forecasted_wind_mwh
n_total_cost = (n_total_capex + n_total_opex)/1000000000

n_total_cost

219.75367514383942

In [ ]:

if years_into_the_future - n_years_to_build > 0:
    n_total_capex = cost_nuclear_plant
    n_mwh_year = 25600000 #3.2GW plant * 8,000,000 MWh/year/GW
else:
    n_total_capex = (years_into_the_future/n_years_to_build)* cost_nuclear_plant
    n_mwh_year = (years_into_the_future/n_years_to_build)*25600000